# Resonator Spectroscopy

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [our tutorials](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/index.html) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a measurement to find the optimal response frequency of a resonator to maximize signal-to-noise for a qubit measurement. The LabOne Q Applications Library provides different experiment workflows to achieve this task. You can either just sweep the frequency of a measure-pulse using the `resonator_spectroscopy` experiment workflow, or perform a 2D sweep to optimize at the same time the amplitude of the measure-pulse (the `resonator_spectroscopy_amplitude` experiment workflow). The measure-pulse modulated at different frequencies will be sent on a readout line traversing multiple resonators each coupled with one qubit.

When isolated, a superconducting resonator can be fully described simply by the magnetic flux threaded through the inductor $\phi$ and the difference $n$ in the number of charges on the two plates of the capacitor in units of Copper pairs. In this case, the resulting Hamiltonian will be

$$H = \frac{E_L}{2}\phi^2 + 4 E_C n^2$$

The above Hamiltonian is changed when the resonator is in contact with an external environment, consisting of coupling to other quantum elements and other undesired interactions with a dissipative bath. This more general Hamiltonian can be simplified in the limit where the resonator and the quantum element are far detuned and there is weak anharmonicity. In this regime, the net effect of this interaction will be a shift in the frequency of both quantum elements due to the Lamb shift. For small couplings, the change in frequency for the resonator will be

$$\omega^{1}_R \approx \omega_R + \frac{g^2}{\Delta}$$

where $g$ is the coupling between the two elements and $\Delta$ is their difference in frequency. This frequency shift effectively allows us to measure the state of a quantum element by checking the response of its coupled resonator. This method is called *dispersive readout*.

![](../../../images/resonators_and_qubits.svg "Sketch of the layout of a simple chip with each qubits coupled to a resonator.")

Given this context, we see now that the scope of this experiment workflow is to characterize the resonator at a particular qubit state, typically when this is in its ground state, by looking at the response in the readout line to different frequencies. In practice, the exact response depends on the details of the circuit. Here is an example of how data would look like when several resonators are connected to the same line:

![](<../../../images/resonator_result.png> "Plot of Resonator spectroscopy where the different frequencies scanned are depicted on the x-axis and the absolute value of the integration is presented in the y-axis.")

### Imports

You'll start by importing the resonator-spectroscopy experiment workflows from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import (
    resonator_spectroscopy,
    resonator_spectroscopy_amplitude,
)
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [getting started tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/getting_started.html).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

Let's first create the options class for the resonator-spectroscopy experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = resonator_spectroscopy.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.SPECTROSCOPY_IQ` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- there is a waiting time of 1$\mu$s between the end of every acquisition and the playback of the subsequent readout pulse (`spectroscopy_reset_delay`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first qubit. 

In [ ]:
# our qubits live here in the demo setup:
qubit = my_platform.qpu.qubits[0]

exp_workflow = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.8e9, 7.2e9, 1001),
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(
    compiled_experiment, length=50e-6, signal_names_to_show=["measure"]
)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `ramsey` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the [Quantum Operations tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html).

In [ ]:
resonator_spectroscopy.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow:

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

### Sweep the amplitude at the Same Time

Getting back to our found parameters, we now repeat the resonator-spectroscopy experiment while sweeping the amplitude at the same time. Since the analysis and the interface are different, a dedicated workflow is used, but the input only differs slightly. Note that, since the amplitude is swept in near-time, the change in amplitude will not be visible in the `OutputSimulator`.

In [ ]:
exp_workflow_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubit=qubit,
    frequencies=np.linspace(6.5e9, 7.5e9, 1001),
    amplitudes=np.linspace(0.1, 0.9, 10),
)

workflow_results_amp = exp_workflow_amp.run()

Great! You've now run your resonator-spectroscopy experiment. Check out other experiments in this manual to keep characterizing your qubits.